# Denton Analysis Tool

The `Denton` Analysis Tool may be used on MMS data to contrust 3d magnetic field vectors.

The main interface is the `Denton` object, which is a subclass of `Kamodo`, a CCMC tool
that provides a functional interface for space weather models and data.


In [1]:
cd AsherCode

/Users/apembrok/git/mms/AsherCode


In [2]:
from Denton import Denton

sDirData = "/Users/apembrok/git/mms/AsherCode/"

denton = Denton(sDirData=sDirData, solve = True)
denton

/Users/apembrok/miniconda4/envs/mms/lib/python3.7/site-packages/kamodo-0.0.2-py3.7.egg/kamodo/readers/__init__.py:7: UserWarning:

TIEGCM reader not installed



Denton([(lambdavec(t),
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (lambdavec,
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (xvec_1(t),
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (xvec_1,
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (xvec_2(t),
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (xvec_2,
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35.499, 35.5  ]))>),
        (xvec_3(t),
         <function Denton.Denton.register_field.<locals>.f(t=array([31.   , 31.001, 31.002, ..., 35.498, 35

# Fieldline visualization

To visualize field lines, we need to solve the initial value problem on $\vec{B}(\vec{x},t)$ given an initial set of seeds points.

## Variable selection

First we'll construct a new kamodo instance so we can focus on just the field manipulation

In [3]:
from kamodo import Kamodo

In [4]:
fields = Kamodo(Bvec = denton.Bvec)
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>)])

## Time selection

Since we are tracing in space and not time, we need a function of the field in time.


In [33]:
t0 = 32.91

In [34]:
from kamodo import kamodofy, pointlike
import numpy as np

@kamodofy
@pointlike(signature = '(n,m)->(n,m)', otypes = [np.float], squeeze = 0)
def bvec(xvec):
    return denton.Bvec(xvec, bvec.t)

bvec.t = t0 

fields['bvec'] = bvec

In [35]:
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (bvec(xvec), <function __main__.bvec(xvec)>),
        (bvec, <function __main__.bvec(xvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(xvec), <function _lambdifygenerated(xvec)>),
        (bhat, <function _lambdifygenerated(xvec)>),
        (bc(s, rvec), <function __main__.boundary(s, rvec)>),
        (bc, <function __main__

Note that $\vec{b}(\vec{x_1(t)})$ should match $\hat{B_1}(t)$

In [36]:
denton.Bvechat_1(t0)

array([ 2.46024091,  0.75626258, -0.22039501])

In [37]:
fields.bvec(denton.xvec_1(t0))

array([ 2.46024091,  0.75626258, -0.22039501])

## Field normalization

To control the spatial resolution of the solver, we'll register a normalized version of the field.

In [14]:
import numpy as np

@kamodofy(equation = "$$\\hat{n}(\\vec{y}) = \\vec{y}/\\sqrt{\\vec{y} \\cdot \\vec{y}} $$")
@pointlike(signature = '(m,n)->(m,n)', squeeze = 0)
def normalized(yvec):   
    r = np.linalg.norm(yvec, axis = 1)
    r[r==0] = np.nan

    try:
        return yvec/r
    except:
        return (yvec.T/r).T
    
fields['nhat'] = normalized
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (bvec(xvec), <function __main__.bvec(xvec)>),
        (bvec, <function __main__.bvec(xvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>)])

Now that we can register a normalized version of the spatial magnetic field.

In [15]:
fields['bhat'] = 'nhat(bvec)'
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (bvec(xvec), <function __main__.bvec(xvec)>),
        (bvec, <function __main__.bvec(xvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(xvec), <function _lambdifygenerated(xvec)>),
        (bhat, <function _lambdifygenerated(xvec)>)])

## Tracer limits

We need to specify a boundary condition for the tracer so it does not go out of bounds. This can be any function of field line length and current position, but it must return positive when inside the bounds and negatie when outide.

In [16]:
from kamodo import event


def boundary(s, rvec):
    """Boundary of the tracer domain"""
    r = np.linalg.norm(rvec)
    
    if np.isnan(r):
        result = 0
    else:
        result = boundary.rmax - r
    return result

# set boundary parameters
boundary.rmax = denton.intro.Lmax
boundary.terminal = True
boundary.direction = 0

In [17]:
fields['bc'] = kamodofy(boundary)
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (bvec(xvec), <function __main__.bvec(xvec)>),
        (bvec, <function __main__.bvec(xvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(xvec), <function _lambdifygenerated(xvec)>),
        (bhat, <function _lambdifygenerated(xvec)>),
        (bc(s, rvec), <function __main__.boundary(s, rvec)>),
        (bc, <function __main__

## Defining Seed points

We will trace from locations of the four MMS spacecraft for this point in time.

In [18]:
t0

32.91

In [19]:
seeds = np.vstack([denton['xvec_{}'.format(isat+1)](t0) for isat in range(4)])
seeds

array([[-0.22609148, -0.42938293,  0.36167197],
       [-0.57525902,  0.33372416, -0.25466636],
       [ 0.16600319,  0.34958773,  0.10506519],
       [ 0.63534731, -0.25392896, -0.21207081]])

## Tracing the field

We now have everything we need to trace the fieldlines. First import the solve method kamodo, which wraps scipy's solve_ivp method.

In [20]:
from kamodo.util import solve

In [21]:
fields['lvec'] = solve(
    fields.bhat,
    seeds, 's',
    (0,3),
    npoints = 15,
    events = fields.bc)

In [22]:
fields

Kamodo([(Bvec(xvec, t),
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (Bvec,
         <function Denton.Denton.register_spatial.<locals>.Bvec(xvec=array([[-3. , -3. , -3. ],
       [-3. , -3. , -2.4],
       [-3. , -3. , -1.8],
       ...,
       [ 3. ,  3. ,  1.8],
       [ 3. ,  3. ,  2.4],
       [ 3. ,  3. ,  3. ]]), t=31.000000000000004)>),
        (bvec(xvec), <function __main__.bvec(xvec)>),
        (bvec, <function __main__.bvec(xvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(xvec), <function _lambdifygenerated(xvec)>),
        (bhat, <function _lambdifygenerated(xvec)>),
        (bc(s, rvec), <function __main__.boundary(s, rvec)>),
        (bc, <function __main__

To get the positions along the fieldline, call $\vec{l}$ with default arguments.

In [23]:

fields.lvec().head()

0         1         2
seed integral                               
0.0  -2.571429 -2.654786 -0.850300  0.985634
     -2.357143 -2.458162 -0.860956  0.900921
     -2.142857 -2.259184 -0.860704  0.821431
     -1.928571 -2.058403 -0.849982  0.747588
     -1.714286 -1.856301 -0.829395  0.679739

## Visualization

We visualize the results using plotly.

In [24]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)

In [25]:
fig = fields.plot('lvec', bhat = dict(xvec = fields.lvec()))

The plot command above generates a figure depicting fieldlines along with associated vectors. However, in order to match the perspective from the paper, we need to modify some of the figure's scene properties:

In [26]:
fig.layout['scene']['camera']['projection']['type'] = 'orthographic'
fig.layout['autosize'] = True
fig.layout.scene.xaxis.title.text = 'L/L_sc'
fig.layout.scene.zaxis.title.text = 'N/N_sc'
fig.layout.scene.xaxis.range = [-2.5,2.5]
fig.layout.scene.yaxis.range = [-2.5,2.5]
fig.layout.scene.zaxis.range = [-2.5,2.5]
# fig.layout['width'] = 1500
# fig.layout['height'] = 700
fig.layout.scene.aspectmode = 'cube'

Save the figure as svg with plotly-orca

!!! note
    You must have plotly-orca installed `conda install -c plotly plotly-orca`

In [27]:
import plotly.io as pio

In [28]:
mkdir docs

In [29]:
mkdir images

In [30]:
pio.write_image(fig, 'images/streamlines3d.svg')

![streamlines](images/streamlines3d.svg?3)

In [31]:
# use iplot for an interactive plot in jupyter notebook
iplot(fig)